In [1]:
import requests
import pandas as pd

# User Inputs
COINS = ["BANANAS31"]              # Coins to fetch
START_DATE = "2025-04-01"           # YYYY-MM-DD
END_DATE = "2025-04-10"             # YYYY-MM-DD
INTERVAL = 5                        # Minutes (1, 5, 15, 30, 60, etc.)

# Map coins to Kraken pairs
COIN_PAIRS = {"BTC": "XXBTZUSD", "ETH": "XETHZUSD"}

# Convert dates to timestamps
start_ts = int(pd.to_datetime(START_DATE).timestamp())
end_ts = int(pd.to_datetime(END_DATE).timestamp())

# Fetch data
url = "https://api.kraken.com/0/public/OHLC"
data_dict = {}

for coin in COINS:
    pair = COIN_PAIRS.get(coin)
    if not pair:
        print(f"Skipping {coin}: No pair found")
        continue

    response = requests.get(url, params={"pair": pair, "interval": INTERVAL, "since": start_ts})

    if response.status_code == 200 and not response.json().get("error"):
        try:
            df = pd.DataFrame(
                response.json()["result"][pair],
                columns=["time", "open", "high", "low", "close", "vwap", "volume", "count"]
            )
            df = df[["time", "close"]].assign(
                time=pd.to_datetime(df["time"], unit="s"),
                close=df["close"].astype(float)
            )
            df = df[(df["time"] >= START_DATE) & (df["time"] <= END_DATE)]
            data_dict[coin] = df.set_index("time")["close"].rename(f"{coin}_close")
        except KeyError:
            print(f"Error for {pair}: Data not found")
    else:
        print(f"Failed for {pair}: Check connection or pair")

# Save to Excel
if data_dict:
    df = pd.concat(data_dict.values(), axis=1, join="inner").reset_index()
    df.columns = ["time"] + [f"{coin}_close" for coin in COINS]
    print("Preview:")
    print(df.head())
    df.to_excel("kraken_btc_eth.xlsx", index=False, sheet_name="BTC_ETH_5min")
    print("Saved to kraken_btc_eth.xlsx")
else:
    print("No data fetched")

Skipping BANANAS31: No pair found
No data fetched
